In [1]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



#from secret import *

In [2]:
def find_url(row):
    Base_Link_01= "https://en.wikipedia.org/wiki/"
    Base_Link_02= "_AFL_season"
    Variable= row
    link = Base_Link_01 + Variable + Base_Link_02
    return (link)

In [30]:
import time
from datetime import datetime

struct_time = time.strptime(str(2017) + "Sunday, 23 August (2:40 pm)", "%Y%A, %d %B (%I:%M %p)")

dt = datetime.fromtimestamp(time.mktime(struct_time))
print(struct_time)
dt

time.struct_time(tm_year=2017, tm_mon=8, tm_mday=23, tm_hour=14, tm_min=40, tm_sec=0, tm_wday=6, tm_yday=235, tm_isdst=-1)


datetime.datetime(2017, 8, 23, 14, 40)

### CONVERT DATE

In [83]:
def date_conversion (x):
    #x = "PF1: Friday, 25 September (6:20 pm)"
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(":")
    if count >1:
        first_colon_index = x.index(":")
        date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)

In [98]:
date_conversion ("PF1: Friday, 25 September (6:20 pm)")

'Friday, 25 September (6:20 pm)'

### MATCH FUNCTION

In [215]:
def match_function(match, this_round, this_year):

    try:


        test = match.find_all('td')
        #open_date = test[0].string
        #print (open_date)
        time_date= date_conversion(test[0].string)
        
        #time_date = test[0].string
        day = time_date.split(',')[0]
        date = time_date.split(',')[1].split('(')[0].strip().split(' ')[0]
        month = time_date.split(',')[1].split('(')[0].strip().split(' ')[1]
        time = (time_date.split('(')[1])[:-1]



        #print (day)
        #print (date)
        #print (month)
        #print (year_)
        #print (time)
        #print (time_date)

        home_team = test[1].find('a').string
        home_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
        home_goals= home_score.split('.')[0]
        home_behinds= home_score.split('.')[1].split('(')[0]
        home_total = (home_score.split('.')[1].split('(')[1])[:-1]
        #print (home_team)
        #print (home_goals)
        #print (home_behinds)
        #print (home_total)



        away_team = test[3].find('a').string
        away_score = (''.join(text for text in test[3].find_all(text=True) if text.parent.name != "a")).strip()
        away_goals= away_score.split('.')[0]
        away_behinds= away_score.split('.')[1].split('(')[0]
        away_total = (away_score.split('.')[1].split('(')[1])[:-1]
        #print (away_team)
        #print (away_goals)
        #print (away_behinds)
        #print (away_total)



        venue = test[4].find('a').string
        venue_crowd = (''.join(text for text in test[4].find_all(text=True) if text.parent.name != "a")).strip()
        venue_crowd_= (venue_crowd.split(':')[1])[:-1].strip()
        final_crowd = venue_crowd_.replace(",", "")
        #print (venue)
        #print (final_crowd)


        match_report_url = test[5].find('a')['href']
        #print (match_report_url)


        ind_match_dict= {
             "Day": day,
             "Date": date,
             "Month": month,
             "Year": this_year,
             "Round": this_round,         
             "Time": time,
             "Home Team": home_team,
             "Home Goals": home_goals,
             "Home Behinds": home_behinds,
             "Home Total": home_total,
             "Away Team": away_team,
             "Away Goals": away_goals,
             "Away Behinds": away_behinds,
             "Away Total": away_total,
             "Venue": venue,
             "Crowd": final_crowd,
             "Away Behinds": away_behinds,
             "Match Link": match_report_url}



        
        return (ind_match_dict)
    
    except Exception as e:
        print("this row does not have match data", e)
        #return False



### ROUND FUNCTION

In [227]:
def round_function(round_number, year_data):    
    games= year_data[round_number].find_all('tr')
    #print (games)
    ## Find the round header
    round_head = (''.join(text for text in games[1].find_all(text=True) if text.parent.name != "a")).strip()
    #current_round = round_number
    ## Only look at matchday tables in each round
    round_matchday = games[2:-1]
    this_round = []
    for match in round_matchday:
        result= match_function(match, round_head, year)
        if result:
            this_round.append(result)
    return (this_round)

### RUN FUNCTION

In [217]:
year_list = ['2017','2016','2015','2014','2013','2012','2011','2010','2009','2008','2007','2006','2005','2004','2003','2002', '2001']

In [231]:
for i in range (3):
    print (i)

0
1
2


### CALL ALL YEAR + THE ROUND DATA AND ADD TO A BIG LIST OF DICTIONARES

In [239]:
year = year_list[2]
round_no = 4
url = find_url(str(year))


## ITERATE YEARS

req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
year_data = soup.find("div", {"id": "mw-content-text"}).find_all('table', {"style": "width: 100%; background-color: #f1f5fc; border: 2px solid #D0E5F5"})

#print (len(year_data))
## ITERATE ROUNDS

for i in range (29):

    this_round = round_function(i, year_data)
    print (this_round)


this row does not have match data list index out of range
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data 'NoneType' object has no attribute 'count'
this row does not have match data list index out of range
this row does not have match data list index out of range
this row does not have match data list index out of range
this row does not have match data list index out of range
this row does not have match data 'NoneType' object has no attribute 'count'
[]
[{'Day': 'Thursday', 'Date': '2', 'Month': 'April', 'Year': '2015', 'Round': 'Round 1 (Opening Round)', 'Time': '7:20\xa0pm', 'Home Team': 'Carlton', 'Home Goals': '11', 'Home Behinds': '12 ', 'Home Total': '78', 'Away Team': 'Richmond', 'Away Goals': '15', 'Away Behinds': '15 ', 'Away Total': '105', 'Venue': 'MCG', 'Crowd': '83493', 'Match Link': 'http://afl.com.au/match-centre/2015/1/carl-v-rich'}, {'Day': 'Saturday', 'Date': '4', 'Month': 'April', 'Year': '2015', 'Roun

### RUNNING

In [129]:
df = pd.DataFrame(this_round)
df.head(100)

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Time,Venue,Year
0,8,11,Geelong,74,27400,22,Friday,8,12,Sydney,80,http://www.afl.com.au/match-centre/2012/13/syd...,June,13,7:50 pm,SCG,2012
1,6,18,Brisbane Lions,114,20146,23,Saturday,14,7,Western Bulldogs,56,http://www.afl.com.au/match-centre/2012/13/wb-...,June,13,1:40 pm,Etihad Stadium,2012
2,10,12,West Coast,82,62957,23,Saturday,13,12,Collingwood,85,http://www.afl.com.au/match-centre/2012/13/col...,June,13,4:40 pm,MCG,2012
3,11,17,Essendon,113,34567,23,Saturday,5,14,Fremantle,89,http://www.afl.com.au/match-centre/2012/13/fre...,June,13,5:40 pm,Patersons Stadium,2012
4,3,9,Greater Western Sydney,57,20070,24,Sunday,15,20,Melbourne,135,http://www.afl.com.au/match-centre/2012/13/mel...,June,13,3:15 pm,MCG,2012
5,11,13,Adelaide,89,18866,24,Sunday,19,17,North Melbourne,121,http://www.afl.com.au/match-centre/2012/13/nmf...,June,13,4:40 pm,Etihad Stadium,2012
